In [7]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [8]:
test=pd.read_csv('test.csv')
train=pd.read_csv('train.csv')

In [9]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [10]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [11]:
train=pd.get_dummies(train)
train= train.fillna(train.mean())

In [19]:
train.isnull().sum().sort_values()

SaleType_New             0
SaleType_Oth             0
SaleType_WD              0
SaleCondition_Abnorml    0
SaleCondition_AdjLand    0
                        ..
YrSold                   0
SalePrice                0
MSZoning_C (all)         0
MSZoning_FV              0
Id                       0
Length: 289, dtype: int64

In [25]:
train_corr=train.corr()
train_corr_sort=train_corr.sort_values('SalePrice',ascending=False)
train_corr_sort['SalePrice'].head(20)

SalePrice           1.000000
OverallQual         0.790982
GrLivArea           0.708624
GarageCars          0.640409
GarageArea          0.623431
TotalBsmtSF         0.613581
1stFlrSF            0.605852
FullBath            0.560664
BsmtQual_Ex         0.553105
TotRmsAbvGrd        0.533723
YearBuilt           0.522897
YearRemodAdd        0.507101
KitchenQual_Ex      0.504094
Foundation_PConc    0.497734
MasVnrArea          0.475241
GarageYrBlt         0.470177
Fireplaces          0.466929
ExterQual_Gd        0.452466
ExterQual_Ex        0.451164
BsmtFinType1_GLQ    0.434597
Name: SalePrice, dtype: float64

In [58]:
cols_train=['OverallQual','GrLivArea','GarageCars','GarageArea','TotalBsmtSF']
X_train_pre = train[cols_train]

# 집 값을 저장합니다.
y = train['SalePrice'].values
x_train, x_test, y_train, y_test = train_test_split(X_train_pre, y, test_size=0.2)

In [55]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [59]:
# 모델의 구조를 설정합니다.
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_dim=x_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.Dense(40, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

# 모델을 실행합니다.
model.compile(optimizer ='adam', loss = 'mean_squared_error')

# 20회 이상 결과가 향상되지 않으면 자동으로 중단되게끔 합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)


# 실행 관련 설정을 하는 부분입니다. 전체의 20%를 검증셋으로 설정합니다.
history = model.fit(x_train, y_train, validation_split=0.25, epochs=2000, batch_size=32, callbacks=[early_stopping_callback])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 10)                60        
                                                                 
 dense_37 (Dense)            (None, 30)                330       
                                                                 
 dense_38 (Dense)            (None, 40)                1240      
                                                                 
 dense_39 (Dense)            (None, 1)                 41        
                                                                 
Total params: 1671 (6.53 KB)
Trainable params: 1671 (6.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/2000
28/28 [==============================] - 3s 11ms/step - loss: 39480197120.0000 - val_loss: 37339643904.0000
Epoch 2/2000
28/28 [============

In [38]:
x_train.shape[1]

8

In [60]:
# 예측 값과 실제 값, 실행 번호가 들어갈 빈 리스트를 만듭니다.
real_prices =[]
pred_prices = []
X_num = []

# 25개의 샘플을 뽑아 실제 값, 예측 값을 출력해 봅니다.
n_iter = 0
Y_prediction = model.predict(x_test).flatten()
for i in range(25):
    real = y_test[i]
    prediction = Y_prediction[i]
    print("실제가격: {:.2f}, 예상가격: {:.2f}".format(real, prediction))
    real_prices.append(real)
    pred_prices.append(prediction)
    n_iter = n_iter + 1
    X_num.append(n_iter)

10/10 [==============================] - 0s 2ms/step
실제가격: 200500.00, 예상가격: 195923.84
실제가격: 148500.00, 예상가격: 156918.72
실제가격: 385000.00, 예상가격: 327176.72
실제가격: 158500.00, 예상가격: 141318.67
실제가격: 154500.00, 예상가격: 130016.76
실제가격: 131000.00, 예상가격: 139040.70
실제가격: 216000.00, 예상가격: 196253.88
실제가격: 127000.00, 예상가격: 127935.23
실제가격: 315500.00, 예상가격: 255274.97
실제가격: 200000.00, 예상가격: 225904.84
실제가격: 90000.00, 예상가격: 102339.98
실제가격: 160000.00, 예상가격: 133157.69
실제가격: 150500.00, 예상가격: 163546.38
실제가격: 101800.00, 예상가격: 96780.44
실제가격: 230000.00, 예상가격: 205192.75
실제가격: 214000.00, 예상가격: 212328.08
실제가격: 162900.00, 예상가격: 197091.98
실제가격: 188700.00, 예상가격: 149589.19
실제가격: 134500.00, 예상가격: 120980.30
실제가격: 130500.00, 예상가격: 173936.47
실제가격: 90000.00, 예상가격: 235804.77
실제가격: 157500.00, 예상가격: 201556.53
실제가격: 136500.00, 예상가격: 134079.14
실제가격: 205000.00, 예상가격: 197078.09
실제가격: 185000.00, 예상가격: 231426.94


In [61]:
import matplotlib as plt
plt.plot(X_num, pred_prices, label='predicted price')
plt.plot(X_num, real_prices, label='real price')
plt.legend()
plt.show()

AttributeError: module 'matplotlib' has no attribute 'plot'